In [ ]:
import torch
import numpy as np
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from datasets import load_dataset
from graph import process_dataset
from IPython.display import clear_output

torch.set_default_device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def encode_graphvc(graph, vertices, vs):  # [0-1] encoded into hvds with precision n
    return bundle(map(lambda n: vs[round(float(n) * len(vs))], graph.nodes))

In [ ]:
class GraphClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, encoder, transformer, step=20):
        self.encoder = encoder
        self.transformer = transformer
        self.step = step
        self.memory = ItemMemory()
        self.vertices = dict()

    def fit(self, X, y):
        classes = {label: [] for label in set(y)}

        for i, graph in enumerate(self.transformer(X)):
            classes[y[i]].append(self.encoder(graph, self.vertices))

        for key, value in classes.items():
            for i in range(0, len(value), self.step):
                H = bundle(value[i : i + self.step])
                self.memory.add_vector(str(key), H)

        return self

    def predict(self, X):
        p = []
        for query in self.transformer(X):
            (label, _, _) = self.memory.cleanup(self.encoder(query, self.vertices))
            p.append(int(label))

        return p

In [ ]:
DATASET = load_dataset("graphs-datasets/MUTAG")["train"]

In [ ]:
(graphs, labels) = process_dataset(DATASET)